# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [5]:
!pip install imutils
from utils import display_image, display_images


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ImportError: cannot import name 'display_image' from 'utils' (C:\Users\kohyu\AppData\Local\Programs\Python\Python312\Lib\site-packages\utils\__init__.py)

In [12]:
# Q1
import cv2 as cv
import imutils

image = cv.imread("images/flower.jfif")

# rotate image by 45 degree
rotated = imutils.rotate_bound(image, angle=45)

display_image("Rotate Image", rotated)

In [18]:
# Q1
import numpy as np

image = cv.imread("images/flower.jfif")

# Get image dimensions
(h, w) = image.shape[:2]
center = (w // 2, h // 2)

# Compute the rotation matrix around the center of the image
angle = 45
scale = 1.0
rotation_matrix = cv.getRotationMatrix2D(center, angle, scale)

# Determine the new dimensions of the image
cos_theta = np.abs(rotation_matrix[0, 0])
sin_theta = np.abs(rotation_matrix[0, 1])
new_w = int((h * sin_theta) + (w * cos_theta))
new_h = int((h * cos_theta) + (w * sin_theta))

# Adjust the rotation matrix to take into account translation
rotation_matrix[0, 2] += (new_w / 2) - center[0]
rotation_matrix[1, 2] += (new_h / 2) - center[1]

# Perform the actual rotation
rotated = cv.warpAffine(image, rotation_matrix, (new_w, new_h))

display_image("Rotate Image", rotated)

In [19]:
# Q2
# Load the images
flower = cv2.imread('images/flower.jfif')
bee = cv2.imread('images/native-bee.png')

# Convert flower image to grayscale
flower_gray = cv2.cvtColor(flower, cv2.COLOR_BGR2GRAY)

# Create a binary mask of the flower
_, mask = cv2.threshold(flower_gray, 90, 255, cv2.THRESH_BINARY)

# Invert the mask
mask_inv = cv2.bitwise_not(mask)

# Define the region of interest (ROI)
rows, cols, _ = flower.shape
roi = bee[0:rows, 0:cols]

# Black-out the area of the flower in the ROI
bee_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

# Take only the flower region from the flower image
flower_fg = cv2.bitwise_and(flower, flower, mask=mask)

# Place the flower in the ROI and modify the main image
dst = cv2.add(bee_bg, flower_fg)
bee[0:rows, 0:cols] = dst

cv2.imwrite('result.png', bee)

display_image('Result', bee)

In [20]:
# Q3
def random_center_crop(image, min_crop_ratio, max_crop_ratio):

    if min_crop_ratio >= max_crop_ratio or min_crop_ratio < 0 or max_crop_ratio > 1:
        raise ValueError("Invalid crop ratio values. min_crop_ratio should be less than max_crop_ratio and between 0.0 and 1.0.")

    height, width = image.shape[:2]

    # Calculate crop sizes 
    min_crop_size = int(min(height, width) * min_crop_ratio)
    max_crop_size = int(min(height, width) * max_crop_ratio)

    # Randomly select crop size
    crop_size = np.random.randint(min_crop_size, max_crop_size + 1)

    # Calculate crop region
    top = (height - crop_size) // 2
    left = (width - crop_size) // 2
    bottom = top + crop_size
    right = left + crop_size

    # Perform crop
    cropped_image = image[top:bottom, left:right]

    return cropped_image

In [21]:
# Q4
def add_salt_and_pepper_noise(image, amount=0.02):

    noisy_image = np.copy(image)

    # Generate random positions to add noise
    num_pixels = int(amount * image.shape[0] * image.shape[1])
    salt_coords = [np.random.randint(0, i - 1, num_pixels) for i in image.shape[:2]]
    pepper_coords = [np.random.randint(0, i - 1, num_pixels) for i in image.shape[:2]]

    # Add salt noise (white pixels)
    noisy_image[salt_coords[0], salt_coords[1]] = 255

    # Add pepper noise (black pixels)
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0

    return noisy_image

# Load sample image 
image = cv2.imread("images/alley_night.jpg")

# Add salt and pepper noise
noisy_image = add_salt_and_pepper_noise(image, amount=0.02)

cv2.imshow('Original Image', image)
cv2.imshow('Noisy Image (Salt and Pepper)', noisy_image)
cv2.waitKey(0)
cv2.destroyAllWindows()